In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Sandbox trading environment

Attributes
----------
instruments
transactions
reference portfolios
set holdings
properties
quotes
aggregation
"""

toggle_code("Toggle Docstring")

## The Challenge

We have a research analyst team who provide recommendations to our portfolio managers. Unfortunately not all of these recommendations lead to a positive outcome. We need a better way to stress test the recommendations of our analyst team in a live environment to ensure that they are robust before they are passed onto our portfolio managers.

We also need to understand what information an analyst was presented with when they made a recommendation. This will allow us to understand the reasoning behind the decision at the time that it was made. This has been very difficult for us to determine previously.

## The Solution

Using LUSID we can construct a "paper portfolio" for each member of our analyst team. Using this paper portfolio they will be able to make trades based on their own recommendations to see how they would have performed under close to real market conditions. We can do this by:

1) Creating a logically separate environment for each analyst

2) Creating our tradeable instrument universe for each analyst

3) Creating a transaction portfolio for each analyst to hold their tradeable instrument universe and creating a reference portfolio for each analyst which contains the index of their tradeable instrument universe and can be used as a benchmark for performance

4) Populating each analyst's portfolio with a starting cash balance

5) Populating each analyst's reference portfolio with the weights for each constituent of their index

6) Allowing our analysts to trade across their tradeable instrument universe and add their transactions to their transaction portfolio

7) Adding the latest prices of our tradeable instrument universe to the market data quote store

8) Valuing our analyst's portfolio using our market data quotes

9) Creating an index from our reference portfolio and building a benchmark for our analyst's to compare their performance against

*To initialise our LUSID environment run the cell below*

In [2]:
# Import Libraries
import pytz
import printer as prettyprint
import pandas as pd
import os

from datetime import datetime, timedelta, time

# Import LUSID
import lusid
import lusid.models as models
import lusid_sample_data as import_data

from lusidjam import RefreshingToken
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

pd.set_option("display.precision", 8)
pd.set_option("display.float_format", "{:0.4f}".format)

print("LUSID Environment Initialised")
print(
    "LUSID API Version: ",
    api_factory.build(lusid.api.ApplicationMetadataApi)
    .get_lusid_versions()
    .build_version,
)

LUSID Environment Initialised
LUSID API Version:  0.6.6621.0


![Scopes](img/paper-lusid.gif)

## 1) Creating a logically separated environment for each analyst

In LUSID we can create separate environments for each of our analysts using scopes. A scope is a container for LUSID objects and can be thought of as a separate identity namespace. Using LUSID's entitlements engine we can ensure that an analyst can only interact with objects inside their scope. Furthermore no other analyst or other stakeholder inside the organisation can access this scope unless explicitly given permission. 

For this example we will create a scope for a single analyst. 

*Run the cell below to initialise our scope. Note that the scope will have a unique 4 character code appended to it to make the name more unique.*

In [3]:
# Fetch our scopes
scope_id = import_data.create_scope_id()
analyst_scope_code = "analyst-paper-{}".format(scope_id)

prettyprint.heading("Analyst Scope Code", analyst_scope_code)

Analyst Scope Code: analyst-paper-396e-176c-9dba-38


*Our LUSID environment now looks like the below. We will update this with each step.*

![Scopes](img/paper-scopes.gif)

## 2) Creating our tradeable instrument universe for each analyst

Before we can set-up our transaction and reference portfolios we need to define the tradeable instrument universe for our analyst. In this case let us assume that the analyst has been tasked with making recommendations to buy or sell equities from the London Stock Exchange FTSE techMark Focus index.

*Run the cell below to import our instrument universe*

In [4]:
# Import our instrument universe from a CSV file using Pandas
instrument_universe = pd.read_csv("data/paper-instruments.csv")
# Look at the first 10 instruments
instrument_universe.head(n=10)

,instrument_name,currency,figi,ticker,isin,sedol,client_internal
0,AVEVA GRP,GBP,BBG000C21Y87,AVV,GB00BBG9VN75,BBG9VN7,imd_56344535
1,BAE SYS.,GBP,BBG000BD5TW4,BA.,GB0002634946,263494,imd_56344983
2,BATM ADVANCED,GBP,BBG000BFJD77,BVC,IL0010849045,911146,imd_56344098
3,BTG,GBP,BBG000BZZK79,BTG,GB0001001592,100159,imd_56344345
4,COBHAM,GBP,BBG000BS6810,COB,GB00B07KD360,B07KD36,imd_56344363
5,COMPUTACENTER,GBP,BBG000BN7CL9,CCC,GB00BV9FP302,BV9FP30,imd_56344296
6,CONSORT MED.,GBP,BBG000BD8278,CSRT,GB0000946276,94627,imd_56344187
7,DIALIGHT,GBP,BBG000BLQNG4,DIA,GB0033057794,3305779,imd_56344572
8,GENUS,GBP,BBG000CTMKX7,GNS,GB0002074580,207458,imd_56344789
9,GRESHAM TECH,GBP,BBG000BS2592,GHT,GB0008808825,880882,imd_56344132


Now that we have our instruments, we can upsert our instruments in a batch. You can read more about upserting instruments in the LUSID API documentation - [LUSID API Docs: Upserting Instruments](https://docs.lusid.com/#operation/UpsertInstruments)

*Run the cell below to upsert our instruments into LUSID*

In [5]:
# Initialise our batch upsert request
batch_upsert_request = {}
# Iterate over our instrument universe
for row, instrument in instrument_universe.iterrows():

    # Set our identifier columns
    identifier_columns = [
        ("isin", "Isin"),
        ("figi", "Figi"),
        ("ticker", "Ticker"),
        ("sedol", "Sedol"),
        ("client_internal", "ClientInternal"),
    ]

    # Create our identifiers
    identifiers = {}
    for identifier in identifier_columns:
        identifiers[identifier[1]] = models.InstrumentIdValue(
            value=instrument[identifier[0]]
        )

    # Add the instrument to our batch request using the FIGI as the main unique identifier
    batch_upsert_request[instrument["instrument_name"]] = models.InstrumentDefinition(
        name=instrument["instrument_name"], identifiers=identifiers
    )

# Call LUSID to upsert our batch
instrument_response = api_factory.build(lusid.api.InstrumentsApi).upsert_instruments(
    request_body=batch_upsert_request
)

# Pretty print the response from LUSID
prettyprint.instrument_response(instrument_response, identifier="Figi")

Instruments Successfully Upserted: 


,Instrument,Figi ID,LUSID Instrument ID
0,RICARDO,BBG000BDYQS3,LUID_AYMZI4LQ
1,CONSORT MED.,BBG000BD8278,LUID_SQNT32VV
2,AVEVA GRP,BBG000C21Y87,LUID_42Q7OH5P
3,MICROGEN,BBG000BDSS88,LUID_C6D5I9LV
4,SPIRENT,BBG000C4LV50,LUID_BMOWRWJ3
5,BATM ADVANCED,BBG000BFJD77,LUID_0XICSR6Q
6,SAGE GRP.,BBG000BN0PP3,LUID_MH9ZRSXY
7,SMITH&NEPHEW,BBG000BF21F2,LUID_ZNNWGYVC
8,PADDY PWR BET,BBG000N8GHT0,LUID_N8CFHDDZ
9,MICRO FOCUS,BBG000G4KJC6,LUID_VU0J71ZX


Our LUSID environment now has an instrument master containing all of our instruments that we will be trading.

![Scopes](img/paper-instrumentmaster.gif)

During the upsertion process you may have noticed that that every instrument that is created in LUSID given a unique LUSID Instrument Id or LUID for short. This ID is used for many methods and is how LUSID uniquely identifies an instrument.

Note that in addition to our instruments, there is also cash in our holdings. Instruments that represent cash are already set-up in LUSID by default. In LUSID the identifier for cash is the ISO4217 currency code. For example British Pounds have an identifier of GBP. 

## 3) Creating a transactions portfolio for each analyst to hold their tradeable instrument universe and creating a reference portfolo for each analyst which contains the index of their tradeable instrument universe and can be used as a benchmark for performance

Now that we have populated our tradeable instrument universe, we need to create our analyst's transaction portfolio for making trades and their reference portfolio for tracking the index. Each portfolio can be referenced by its unique 'code'. Let us define the code for our two portfolios.

*Run the cell below to define our portfolio codes*

In [6]:
# Define unique codes for each of our portfolios
transaction_portfolio_code = "LSE_FTSE_techMarkFocus_transactions"
reference_portfolio_code = "LSE_FTSE_techMarkFocus_reference"
# Pretty print our codes
prettyprint.heading("Transaction Portfolio Code", transaction_portfolio_code)
prettyprint.heading("Reference Portfolio Code", reference_portfolio_code)

Transaction Portfolio Code: LSE_FTSE_techMarkFocus_transactions
Reference Portfolio Code: LSE_FTSE_techMarkFocus_reference


Now that we have the code for our portfolios we can go ahead and create them in LUSID. Let us first create our transaction portfolio. Note that we will create both portfolios as being available as of 3 days ago. This will allow us to add backdated transactions and compute backdated valuations.

You can read more about creating transaction portfolios in the LUSID API documentation: [LUSID API Docs: Creating Transaction Portfolios](https://docs.lusid.com/#operation/CreatePortfolio)

*Run the cell below to create our transaction portfolio*

In [7]:
# The date our portfolios were first created
portfolio_creation_date = (datetime.now(pytz.UTC) - timedelta(days=3)).isoformat()

# Create the request to add our portfolio
transaction_portfolio_request = models.CreateTransactionPortfolioRequest(
    display_name=transaction_portfolio_code,
    code=transaction_portfolio_code,
    base_currency="GBP",
    description="Paper transaction portfolio",
    created=portfolio_creation_date,
)

# Call LUSID to create our portfolio
portfolio_response = api_factory.build(
    lusid.api.TransactionPortfoliosApi
).create_portfolio(
    scope=analyst_scope_code,
    create_transaction_portfolio_request=transaction_portfolio_request,
)

# Pretty print the response from LUSID
prettyprint.portfolio_response(portfolio_response)

Portfolio Created
Scope: analyst-paper-396e-176c-9dba-38
Code: LSE_FTSE_techMarkFocus_transactions
Portfolio Effective From: 2021-03-20 15:07:36.407320+00:00
Portfolio Created On: 2021-03-23 15:07:36.683771+00:00



![Scopes](img/paper-transactionsportfolio.gif)

Now that we have created our transaction portfolio we can also create our referene portfolio to track the techMarkFocus index. 

You can read more about creating reference portfolios in the LUSID API documentation: [LUSID API Docs: Creating Reference Portfolios](https://docs.lusid.com/#operation/CreateReferencePortfolio)

*Run the cell below to create our reference portfolio*

In [8]:
# The date our portfolios were first created
portfolio_creation_date = (datetime.now(pytz.UTC) - timedelta(days=3)).isoformat()

# Create the request to add our portfolio
reference_portfolio_request = models.CreateReferencePortfolioRequest(
    display_name=reference_portfolio_code,
    code=reference_portfolio_code,
    description="Paper reference portfolio",
    created=portfolio_creation_date,
)

# Call LUSID to create our reference portfolio
portfolio_response = api_factory.build(
    lusid.api.ReferencePortfolioApi
).create_reference_portfolio(
    scope=analyst_scope_code,
    create_reference_portfolio_request=reference_portfolio_request,
)

# Pretty print our response from LUSID
prettyprint.portfolio_response(portfolio_response)

Portfolio Created
Scope: analyst-paper-396e-176c-9dba-38
Code: LSE_FTSE_techMarkFocus_reference
Portfolio Effective From: 2021-03-20 15:07:36.827246+00:00
Portfolio Created On: 2021-03-23 15:07:37.053632+00:00



Our LUSID environment is now populated with our two portfolios. One for our analyst to make trades across and the other to track the weights of the index of the tradeable instrument universe to produce a benchmark.

![Scopes](img/paper-referenceportfolio.gif)

## 4) Populating our analyst's portfolio with a starting cash balance

To set our starting cash balance we use LUSID's 'Set Holdings' capability. This allows us to set the holdings of a portfolio to a given state. In this case we will add in a starting cash balance. Let us give the analyst a starting cash balance of £30,000,000. We will make this balance effective from just after the creation of our portfolio. 

*Run the cell below to set our starting cash balance*

In [9]:
# Set the date from which the cash balance will apply to be just after portfolio creation
holdings_effective_date = (datetime.now(pytz.UTC) - timedelta(days=2.9)).isoformat()
# Define our initial cash balance
initial_cash_balance = 30000000

# Create a holding adjustment to set our initial cash balance
holding_adjustment = [
    models.AdjustHoldingRequest(
        instrument_identifiers={"Instrument/default/Currency": "GBP"},
        tax_lots=[
            models.TargetTaxLotRequest(
                units=initial_cash_balance,
                cost=models.CurrencyAndAmount(
                    amount=initial_cash_balance, currency="GBP"
                ),
                portfolio_cost=initial_cash_balance,
                price=1,
            )
        ],
    )
]

# Call LUSID to set our initial cash balance
set_holdings_response = api_factory.build(
    lusid.api.TransactionPortfoliosApi
).set_holdings(
    scope=analyst_scope_code,
    code=transaction_portfolio_code,
    effective_at=holdings_effective_date,
    adjust_holding_request=holding_adjustment,
)

# Pretty print our response from LUSID
prettyprint.set_holdings_response(
    set_holdings_response, analyst_scope_code, transaction_portfolio_code
)

Holdings Successfully Set for Portfolio
Scope: analyst-paper-396e-176c-9dba-38
Code: LSE_FTSE_techMarkFocus_transactions
Holdings Effective From: 2021-03-20 15:07:36.407320+00:00
Holdings Created On: 2021-03-23 15:07:37.620892+00:00



Our transactions portfolio is now populated with an initial cash balance and is no longer empty.

![Scopes](img/paper-transactionsportfoliofilled.gif)

## 5) Populating our reference portfolio with the correct weights so that it tracks the index as closely as possible

To set up our reference portfolio we need to initialise it with the constituents that make up the index as well as their weight. In this case we are going to weight the consitutents based on their market capitalisation. We will import their market capitalisation from a CSV file generated from one of our internal systems. We will also configure our reference portfolio to be updated once a quarter. We could update this more frequently if we wished. 

*Run the cell below to import our market capitalisation figures for our instruments*

In [10]:
# Import the market capitalisation of each of our instruments in the index
instrument_market_cap = pd.read_csv("data/paper-weights.csv")
# Take a look at the first 10 market capitalisations
instrument_market_cap.head(n=10)

,instrument_name,currency,figi,ticker,isin,sedol,marketcap
0,AVEVA GRP,GBP,BBG000C21Y87,AVV,GB00BBG9VN75,BBG9VN7,4502.6800
1,BAE SYS.,GBP,BBG000BD5TW4,BA.,GB0002634946,0263494,16349.6600
2,BATM ADVANCED,GBP,BBG000BFJD77,BVC,IL0010849045,911146,185.2500
3,BTG,GBP,BBG000BZZK79,BTG,GB0001001592,100159,3202.2000
4,COBHAM,GBP,BBG000BS6810,COB,GB00B07KD360,B07KD36,2579.9000
5,COMPUTACENTER,GBP,BBG000BN7CL9,CCC,GB00BV9FP302,BV9FP30,1161.9600
6,CONSORT MED.,GBP,BBG000BD8278,CSRT,GB0000946276,0094627,422.5600
7,DIALIGHT,GBP,BBG000BLQNG4,DIA,GB0033057794,3305779,133.3900
8,GENUS,GBP,BBG000CTMKX7,GNS,GB0002074580,0207458,1455.7200
9,GRESHAM TECH,GBP,BBG000BS2592,GHT,GB0008808825,0880882,64.3400


Now that we have our market capitalisation figures we can upsert our constituents into the reference portfolio. We can add the weights in any format and LUSID will normalise them to equal 1. 

You can read more about upserting constituents in the LUSID API documentation: [LUSID API Docs: Upserting Constituents to a Reference Portfolio](https://docs.lusid.com/#operation/UpsertReferencePortfolioConstituents)

*Run the cell below to upsert our constituents*

In [11]:
# Initialise a list to hold our constituents
constituents = []
# Work out the total market capitalisation of the entire index
total = instrument_market_cap["marketcap"].sum()

# Iterate over instrument unvierse to add each constituent to our list
for row in instrument_market_cap.iterrows():
    # Collect our instrument
    instrument = row[1]
    # Calculate our constituents weight based on market cap and add it to our list
    constituents.append(
        models.ReferencePortfolioConstituentRequest(
            instrument_identifiers={"Instrument/default/Figi": instrument["figi"]},
            weight=instrument["marketcap"] / total,
            currency=instrument["currency"],
        )
    )

# Create our request to add our constituents
constituents_request = models.UpsertReferencePortfolioConstituentsRequest(
    effective_from=holdings_effective_date,
    weight_type="Periodical",
    period_type="Quarterly",
    period_count=4,
    constituents=constituents,
)

# Call LUSID to upsert our constituents into our reference portfolio
response = api_factory.build(
    lusid.api.ReferencePortfolioApi
).upsert_reference_portfolio_constituents(
    scope=analyst_scope_code,
    code=reference_portfolio_code,
    upsert_reference_portfolio_constituents_request=constituents_request,
)

print("Constituents Upserted")

Constituents Upserted


Our reference portfolio is no longer empty and now contains the weights for each instrument in our index against which the analyst's performance will be compared.

![Scopes](img/paper-referenceportfoliofilled.gif)

## 6) Allow our analysts to trade across their tradeable instrument universe and add their transactions to their transaction portfolio

Now that we have set up our portfolios we can add in our analyst's trades. These would likel be populated from an order management or similar system. In this case we are going to import the trades over the last two days from a CSV file.

*Run the cell below to import our transactions*

In [12]:
# Import our analyst's trades from a CSV file
analyst_transactions = import_data.fetch_client_transactions(
    "paper-transactions.csv", days_back=2
)

# Pretty print their activity
analyst_transactions.head(n=20)

,transaction_id,type,portfolio,instrument_name,instrument_uid,transaction_date,settlement_date,units,transaction_price,transaction_currency,total_cost,strategy,description
0,tid_124398219481,Buy,LSE_FTSE_techMarkFocus_transactions,AVEVA GRP,BBG000C21Y87,2021-03-21T08:32:52.932183+00:00,2021-03-23T08:32:52.932183+00:00,265600,28.9400,GBP,7686464.0000,quantitativeSignal,Purchase
1,tid_339423984894,FundsOut,LSE_FTSE_techMarkFocus_transactions,GBP_Cash,GBP,2021-03-21T14:00:20.810434+00:00,2021-03-23T14:00:20.810434+00:00,265600,0.0058,GBP,1537.2928,quantitativeSignal,Brokerage Fees
2,tid_339423984896,FundsOut,LSE_FTSE_techMarkFocus_transactions,GBP_Cash,GBP,2021-03-22T13:50:43.929637+00:00,2021-03-24T13:50:43.929637+00:00,265600,0.0145,GBP,3843.2320,quantitativeSignal,Stamp Duty
3,tid_359231290933,Buy,LSE_FTSE_techMarkFocus_transactions,BATM ADVANCED,BBG000BFJD77,2021-03-22T10:08:44.090896+00:00,2021-03-24T10:08:44.090896+00:00,750000,0.4630,GBP,347250.0000,fundamentalAnalysis,Purchase
4,tid_223239424244,FundsOut,LSE_FTSE_techMarkFocus_transactions,GBP_Cash,GBP,2021-03-22T13:13:04.229791+00:00,2021-03-24T13:13:04.229791+00:00,750000,0.0001,GBP,69.4500,fundamentalAnalysis,Brokerage Fees
5,tid_354242442211,FundsOut,LSE_FTSE_techMarkFocus_transactions,GBP_Cash,GBP,2021-03-21T12:48:13.117522+00:00,2021-03-23T12:48:13.117522+00:00,750000,0.0002,GBP,173.6250,fundamentalAnalysis,Stamp Duty
6,tid_342398989895,SellShort,LSE_FTSE_techMarkFocus_transactions,COMPUTACENTER,BBG000BN7CL9,2021-03-21T11:32:18.692157+00:00,2021-03-23T11:32:18.692157+00:00,150000,10.5000,GBP,1575000.0000,fundamentalAnalysis,Purchase
7,tid_384809092344,FundsOut,LSE_FTSE_techMarkFocus_transactions,GBP_Cash,GBP,2021-03-21T10:20:16.435977+00:00,2021-03-23T10:20:16.435977+00:00,150000,0.0021,GBP,315.0000,fundamentalAnalysis,Brokerage Fees
8,tid_348234938244,FundsOut,LSE_FTSE_techMarkFocus_transactions,GBP_Cash,GBP,2021-03-22T14:28:04.626371+00:00,2021-03-24T14:28:04.626371+00:00,150000,0.0053,GBP,787.5000,fundamentalAnalysis,Stamp Duty
9,tid_239394829484,Buy,LSE_FTSE_techMarkFocus_transactions,DIALIGHT,BBG000BLQNG4,2021-03-21T08:32:46.526003+00:00,2021-03-23T08:32:46.526003+00:00,1500000,4.1800,GBP,6270000.0000,quantitativeSignal,Purchase


We can set custom properties on our transactions. In this case we are going to create a strategy property which allows the analyst to note what strategy they were following when they made the trade. These will help us justify their trades and recommendations when it comes to making them for real. 

You can read more about creating properties in the LUSID API documentation: [LUSID API Docs: Defining a new Property](https://docs.lusid.com/#operation/CreatePropertyDefinition)

*Run the cell below to create our transaction strategy property*

In [13]:
# Create a request to define our strategy property
property_request = models.CreatePropertyDefinitionRequest(
    domain="Transaction",
    scope=analyst_scope_code,
    code="strategy",
    value_required=False,
    display_name="strategy",
    data_type_id=models.ResourceId(scope="system", code="string"),
)

# Call LUSID to create our new property
property_response = api_factory.build(
    lusid.api.PropertyDefinitionsApi
).create_property_definition(create_property_definition_request=property_request)

# Grab the key off the response to use when referencing this property in other LUSID calls
strategy_property_key = property_response.key

# Pretty print our strategy property key
prettyprint.heading("Strategy Property Key: ", strategy_property_key)

Strategy Property Key: : Transaction/analyst-paper-396e-176c-9dba-38/strategy


Now that we have our trades imported and our property created we can upsert them into LUSID. Read more about adding transactions in the LUSID API documentation: [LUSID API Docs: Upserting Transactions](https://docs.lusid.com/#operation/UpsertTransactions)

*Run the cell below to upsert our transactions into our transactions portfolio*

In [14]:
# Initialise a list to hold our transactions
batch_transaction_requests = []

# Iterate over the transactions for each portfolio
for index, transaction in analyst_transactions.iterrows():

    if "Cash" in transaction["instrument_name"]:
        identifier_key = "Instrument/default/Currency"
    else:
        identifier_key = "Instrument/default/Figi"

    batch_transaction_requests.append(
        models.TransactionRequest(
            transaction_id=transaction["transaction_id"],
            type=transaction["type"],
            instrument_identifiers={identifier_key: transaction["instrument_uid"]},
            transaction_date=transaction["transaction_date"],
            settlement_date=transaction["settlement_date"],
            units=transaction["units"],
            transaction_price=models.TransactionPrice(
                price=transaction["transaction_price"], type="Price"
            ),
            total_consideration=models.CurrencyAndAmount(
                amount=transaction["total_cost"],
                currency=transaction["transaction_currency"],
            ),
            source="Client",
            transaction_currency=transaction["transaction_currency"],
            properties={
                strategy_property_key: models.PerpetualProperty(
                    key=strategy_property_key,
                    value=models.PropertyValue(label_value=transaction["strategy"]),
                )
            },
        )
    )

# Call LUSID to upsert our transactions
transaction_response = api_factory.build(
    lusid.api.TransactionPortfoliosApi
).upsert_transactions(
    scope=analyst_scope_code,
    code=transaction_portfolio_code,
    transaction_request=batch_transaction_requests,
)

# Pretty print the response from LUSID
prettyprint.transactions_response(
    transaction_response, analyst_scope_code, transaction_portfolio_code
)

Transactions Successfully Upserted into Portfolio
Scope: analyst-paper-396e-176c-9dba-38
Code: LSE_FTSE_techMarkFocus_transactions
Transactions Effective From: 2021-03-22 16:09:02.940532+00:00
Transactions Created On: 2021-03-23 15:07:39.029724+00:00



Our transaction portfolio is now populated with our analyst's paper trades.

## 7) Upload market data quotes to hold the latest prices of our tradeable instrument universe

To value our portfolios we need current prices. There are a number of different ways and sources that we can get our pricing information. In this case we are going to load the prices when we set up our portfolios and the current prices from a CSV file.

*Run the cell below to import our pricing information*

In [15]:
# Import our instrument prices from a CSV file
instrument_prices = pd.read_csv("data/paper-analytics.csv")
# Pretty print our pricing
instrument_prices.head(n=10)

,instrument_name,currency,figi,price_original,price_current,ticker,isin,sedol
0,AVEVA GRP,GBP,BBG000C21Y87,27.8400,30.3178,AVV,GB00BBG9VN75,BBG9VN7
1,BAE SYS.,GBP,BBG000BD5TW4,5.1540,4.7829,BA.,GB0002634946,263494
2,BATM ADVANCED,GBP,BBG000BFJD77,0.4590,0.4246,BVC,IL0010849045,911146
3,BTG,GBP,BBG000BZZK79,8.2750,8.9701,BTG,GB0001001592,100159
4,COBHAM,GBP,BBG000BS6810,1.0715,1.0222,COB,GB00B07KD360,B07KD36
5,COMPUTACENTER,GBP,BBG000BN7CL9,10.6800,11.4596,CCC,GB00BV9FP302,BV9FP30
6,CONSORT MED.,GBP,BBG000BD8278,8.7000,9.3090,CSRT,GB0000946276,94627
7,DIALIGHT,GBP,BBG000BLQNG4,3.9200,3.5515,DIA,GB0033057794,3305779
8,GENUS,GBP,BBG000CTMKX7,22.4600,20.5958,GNS,GB0002074580,207458
9,GRESHAM TECH,GBP,BBG000BS2592,0.9450,1.0008,GHT,GB0008808825,880882


Now that we have our pricing information we can store this data inside LUSID in the quote store. We will create two stores one for the prices from 3 days ago when we originally set-up the portfolios and one for the current prices.

You can read more about upserting quotes LUSID API documentation: [LUSID API Docs: Upsert Quotes](https://www.lusid.com/docs/api/#operation/UpsertQuotes)

*Run the cell below to upsert quotes*

In [16]:
# Set our quotes effective dates
quotes_effective_date = datetime.now(pytz.UTC) - timedelta(days=3)
today = datetime.now(pytz.UTC)

instrument_quotes = {}

# Create prices for all instruments except cash
for index, instrument in instrument_prices.iterrows():

    if "Cash" in instrument["instrument_name"]:
        continue

    # Get our Lusid Instrument Id
    luid = (
        api_factory.build(lusid.api.InstrumentsApi)
        .get_instrument(identifier_type="Figi", identifier=instrument["figi"])
        .lusid_instrument_id
    )

    instrument_quotes[luid + str(quotes_effective_date)] = models.UpsertQuoteRequest(
        quote_id=models.QuoteId(
            quote_series_id=models.QuoteSeriesId(
                provider="DataScope",
                instrument_id=luid,
                instrument_id_type="LusidInstrumentId",
                quote_type="Price",
                field="Mid",
            ),
            effective_at=quotes_effective_date.isoformat()
        ),
        metric_value=models.MetricValue(
            value=instrument["price_original"], unit=instrument["currency"]
        ),
        lineage="InternalSystem",
    )

    instrument_quotes[luid + str(today)] = models.UpsertQuoteRequest(
        quote_id=models.QuoteId(
            quote_series_id=models.QuoteSeriesId(
                provider="DataScope",
                instrument_id=luid,
                instrument_id_type="LusidInstrumentId",
                quote_type="Price",
                field="Mid",
            ),
            effective_at=today.isoformat()
        ),
        metric_value=models.MetricValue(
            value=instrument["price_current"], unit=instrument["currency"]
        ),
        lineage="InternalSystem",
    )

response = api_factory.build(lusid.api.QuotesApi).upsert_quotes(
    scope=analyst_scope_code, request_body=instrument_quotes
)

prettyprint.upsert_quotes_response(response)

,_lineage,_cut_label,_uploaded_by,_as_at,_scale_factor,discriminator,_provider,_price_source,_instrument_id,_instrument_id_type,_quote_type,_field,_value,_unit,status
0,InternalSystem,,00u97x74rdncdc85w2p7,2021-03-23 15:07:41.254732+00:00,None,None,DataScope,None,LUID_42Q7OH5P,LusidInstrumentId,Price,Mid,27.8400,GBP,Success
1,InternalSystem,,00u97x74rdncdc85w2p7,2021-03-23 15:07:41.254732+00:00,None,None,DataScope,None,LUID_42Q7OH5P,LusidInstrumentId,Price,Mid,30.3178,GBP,Success
2,InternalSystem,,00u97x74rdncdc85w2p7,2021-03-23 15:07:41.254732+00:00,None,None,DataScope,None,LUID_9UU2Z82D,LusidInstrumentId,Price,Mid,5.1540,GBP,Success
3,InternalSystem,,00u97x74rdncdc85w2p7,2021-03-23 15:07:41.254732+00:00,None,None,DataScope,None,LUID_9UU2Z82D,LusidInstrumentId,Price,Mid,4.7829,GBP,Success
4,InternalSystem,,00u97x74rdncdc85w2p7,2021-03-23 15:07:41.254732+00:00,None,None,DataScope,None,LUID_0XICSR6Q,LusidInstrumentId,Price,Mid,0.4590,GBP,Success
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,InternalSystem,,00u97x74rdncdc85w2p7,2021-03-23 15:07:41.254732+00:00,None,None,DataScope,None,LUID_87GBS9VC,LusidInstrumentId,Price,Mid,0.7022,GBP,Success
58,InternalSystem,,00u97x74rdncdc85w2p7,2021-03-23 15:07:41.254732+00:00,None,None,DataScope,None,LUID_4P2OOVK4,LusidInstrumentId,Price,Mid,1.5480,GBP,Success
59,InternalSystem,,00u97x74rdncdc85w2p7,2021-03-23 15:07:41.254732+00:00,None,None,DataScope,None,LUID_4P2OOVK4,LusidInstrumentId,Price,Mid,1.6780,GBP,Success
60,InternalSystem,,00u97x74rdncdc85w2p7,2021-03-23 15:07:41.254732+00:00,None,None,DataScope,None,LUID_9011RJ64,LusidInstrumentId,Price,Mid,20.8000,GBP,Success


Our LUSID environment now looks like the below. We have a reference portfolio to hold our index which will act as a benchmark, we have a transaction portfolio for our analyst to make paper trades and we have a quotes store to value our portfolios.

![Scopes](img/paper-quotestore.gif)

## 8) Value each portfolio using our market data quotes

We can now value our portfolios to see if the analyst has been making good decisions. Let us start by valuing our transaction portfolio to find out what our absolute return is across our tradeable instrument universe. 

We do this by making an aggregation request to LUSID. We will aggregate by instrument against the number of units, cost, present value and price. 

You can read more about aggregation in the LUSID API documentation: [LUSID API Docs: Aggregate Data in a Portfolio](https://docs.lusid.com/#operation/GetAggregationByPortfolio)

*Run the cell below to make our aggregation request on our transaction portfolio and see how our analyst is performing*

In [17]:
recipe_scope = "sandbox"
recipe_code = "market_value"

# Create and upsert a recipe
configuration_recipe_api = api_factory.build(lusid.api.ConfigurationRecipeApi)

configuration_recipe = models.ConfigurationRecipe(
    scope=recipe_scope,
    code=recipe_code,
    market=models.MarketContext(
        market_rules=[
            models.MarketDataKeyRule(
                key="Equity.LusidInstrumentId.*",
                supplier="DataScope",
                data_scope=analyst_scope_code,
                quote_type="Price",
                field="Mid",
            )
        ],
        suppliers=models.MarketContextSuppliers(
            commodity="DataScope",
            credit="DataScope",
            equity="DataScope",
            fx="DataScope",
            rates="DataScope",
        ),
        options=models.MarketOptions(
            default_supplier="DataScope",
            default_instrument_code_type="LusidInstrumentId",
            default_scope=analyst_scope_code,
        ),
    ),
)

upsert_configuration_recipe_response = (
    configuration_recipe_api.upsert_configuration_recipe(
        upsert_recipe_request=models.UpsertRecipeRequest(
            configuration_recipe=configuration_recipe
        )
    )
)

# Create the valuation request
valuation_request = models.ValuationRequest(
    recipe_id=models.ResourceId(scope=recipe_scope, code=recipe_code),
    metrics=[
        models.AggregateSpec(key="Instrument/default/LusidInstrumentId", op="Value"),
        models.AggregateSpec(key="Instrument/default/Name", op="Value"),
        models.AggregateSpec(key="Holding/default/Units", op="Sum"),
        models.AggregateSpec(key="Holding/default/Cost", op="Sum"),
        models.AggregateSpec(key="Holding/default/PV", op="Sum"),
        models.AggregateSpec(key="Holding/default/Price", op="Sum"),
    ],
    group_by=["Instrument/default/LusidInstrumentId"],
    portfolio_entity_ids=[
        models.PortfolioEntityId(
            scope=analyst_scope_code, code=transaction_portfolio_code
        )
    ],
    valuation_schedule=models.ValuationSchedule(effective_at=today.isoformat()),
)

# Perform a valuation
valuation = api_factory.build(lusid.api.AggregationApi).get_valuation(
    valuation_request=valuation_request
)

df = pd.DataFrame(valuation.data)
df.loc["Total"] = df.sum(numeric_only=True)
df["return"] = (df["Sum(Holding/default/PV)"] - df["Sum(Holding/default/Cost)"]) / df[
    "Sum(Holding/default/Cost)"
]
df

,Instrument/default/LusidInstrumentId,Instrument/default/Name,Sum(Holding/default/Units),Sum(Holding/default/Cost),Sum(Holding/default/PV),Sum(Holding/default/Price),return
0,CCY_GBP,GBP,4364776.0000,4364776.0000,4364776.0000,19.0000,0.0000
1,LUID_B5KMMX6J,DIALIGHT,1500000.0000,6270000.0000,5327280.0000,3.5515,-0.1504
2,LUID_42Q7OH5P,AVEVA GRP,265600.0000,7686464.0000,8052397.0560,30.3178,0.0476
3,LUID_9011RJ64,XP POWER,98000.0000,2156000.0000,1912019.2000,19.5104,-0.1132
4,LUID_QKL1C8AY,SDL,55000.0000,305250.0000,308154.0000,5.6028,0.0095
5,LUID_0TTPI8L4,COMPUTACENTER,-150000.0000,-1575000.0000,-1718946.0000,11.4596,0.0914
6,LUID_C6D5I9LV,MICROGEN,234000.0000,917280.0000,843094.9800,3.6030,-0.0809
7,LUID_ECY4Y5DI,ULTRA ELEC.,120000.0000,1582800.0000,1603546.8000,13.3629,0.0131
8,LUID_MH9ZRSXY,SAGE GRP.,-98000.0000,-577220.0000,-643432.7200,6.5656,0.1147
9,LUID_0XICSR6Q,BATM ADVANCED,750000.0000,347250.0000,318431.2500,0.4246,-0.0830


## 9) Compare performance across the analyst's reference and transaction portfolios

Now that we have got our analyst's absolute return we want to see how this compares to their benchmark. In LUSID we can't value our reference portfolio directly as all it contains is weights. What we can is decide on an arbitary index level (in this case we will start with 1000), create a new transaction portfolio to hold our index and then populate it based on the weights from our reference portfolio.

First things first, let us give our index portfolio a code.

*Run the cell below to create a unique code for our index portfolio*

In [18]:
index_portfolio_code = "LSE_FTSE_techMarkFocus_index"
prettyprint.heading("Index Portfolio Code", index_portfolio_code)

Index Portfolio Code: LSE_FTSE_techMarkFocus_index


Now we can create our index portfolio

*Run the cell below to create our index portfolio*

In [19]:
# Create the request to add our portfolio
transaction_portfolio_request = models.CreateTransactionPortfolioRequest(
    display_name=index_portfolio_code,
    code=index_portfolio_code,
    base_currency="GBP",
    description="Index transaction portfolio",
    created=portfolio_creation_date,
)

# Create our portfolio
portfolio_response = api_factory.build(
    lusid.api.TransactionPortfoliosApi
).create_portfolio(
    scope=analyst_scope_code,
    create_transaction_portfolio_request=transaction_portfolio_request,
)

prettyprint.portfolio_response(portfolio_response)

Portfolio Created
Scope: analyst-paper-396e-176c-9dba-38
Code: LSE_FTSE_techMarkFocus_index
Portfolio Effective From: 2021-03-20 15:07:36.827246+00:00
Portfolio Created On: 2021-03-23 15:07:42.290247+00:00



We now have an index portfolio that we can use to hold our index and obtain our benchmark.

![Scopes](img/paper-indexportfolio.gif)

In [20]:
# Set an arbitary index level to start our index with
index_level = 1000
# Call LUSID - get the constituents of our index from our reference portfolio
constituents = api_factory.build(
    lusid.api.ReferencePortfolioApi
).get_reference_portfolio_constituents(
    scope=analyst_scope_code,
    code=reference_portfolio_code,
    effective_at=datetime.now(pytz.UTC)).isoformat()
# Initialise our list to hold the adjustments we need to make to our index to set it up
index_setup = []
# Get our weights from the constituents into a better format to work with
weights = {
    constituent.instrument_uid: constituent.weight
    for constituent in constituents.constituents
}

# Iterate over our prices
for index, instrument in instrument_prices.iterrows():

    # Get our Lusid Instrument ID
    Luid = (
        api_factory.build(lusid.api.InstrumentsApi)
        .get_instrument(identifier_type="Figi", identifier=instrument["figi"])
        .lusid_instrument_id
    )
    # Get the initial price for each constituent
    inception_price = instrument["price_original"]
    # Work out how much of the index this constituent should make up using its w
    index_cost = weights[Luid] * index_level
    # Work out how many units we should therefore buy
    index_units = index_cost / inception_price
    # Create our request for this instrument
    index_setup.append(
        models.AdjustHoldingRequest(
            instrument_identifiers={"Instrument/default/Figi": instrument["figi"]},
            tax_lots=[
                models.TargetTaxLotRequest(
                    units=index_units,
                    cost=models.CurrencyAndAmount(amount=index_cost, currency="GBP"),
                    portfolio_cost=index_cost,
                    price=inception_price,
                )
            ],
        )
    )
# Call LUSID to set all of our holdings to the initial index level
adjust_holdings_response = api_factory.build(
    lusid.api.TransactionPortfoliosApi
).set_holdings(
    scope=analyst_scope_code,
    code=index_portfolio_code,
    effective_at=portfolio_creation_date,
    adjust_holding_request=index_setup,
)

prettyprint.set_holdings_response(
    adjust_holdings_response, analyst_scope_code, index_portfolio_code
)

Holdings Successfully Set for Portfolio
Scope: analyst-paper-396e-176c-9dba-38
Code: LSE_FTSE_techMarkFocus_index
Holdings Effective From: 2021-03-20 15:07:36.827246+00:00
Holdings Created On: 2021-03-23 15:07:44.753711+00:00



We now have an index portfolio that we can value.

![Scopes](img/paper-indexportfoliofilled.gif)

In [21]:
# Create the valuation request
valuation_request = models.ValuationRequest(
    recipe_id=models.ResourceId(scope=recipe_scope, code=recipe_code),
    metrics=[
        models.AggregateSpec(key="Portfolio/default/Name", op="Value"),
        models.AggregateSpec(key="Holding/default/PV", op="Sum"),
        models.AggregateSpec(key="Holding/default/Cost", op="Sum"),
    ],
    group_by=["Portfolio/default/Name"],
    portfolio_entity_ids=[
        models.PortfolioEntityId(scope=analyst_scope_code, code=index_portfolio_code)
    ],
    valuation_schedule=models.ValuationSchedule(effective_at=today.isoformat()),
)

# Perform a valuation
valuation = api_factory.build(lusid.api.AggregationApi).get_valuation(
    valuation_request=valuation_request
)

# Pretty print the response from LUSID
df = pd.DataFrame(valuation.data)
df["return"] = (df["Sum(Holding/default/PV)"] - df["Sum(Holding/default/Cost)"]) / df[
    "Sum(Holding/default/Cost)"
]
df = df.rename(
    columns={
        "Sum(Holding/default/PV)": "Current Index Level",
        "Sum(Holding/default/Cost)": "Initial Index Level",
    }
)
df

,Portfolio/default/Name,Current Index Level,Initial Index Level,return
0,LSE_FTSE_techMarkFocus_index,995.9674,999.9800,-0.0040
